In [12]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [13]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [14]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [ ]:
dataset = generate_dataset()

# Commenting this out so as not to overwrite generated dataset
# with open("aws_prompt_evaluation_dataset.json", "w") as f:
#     json.dump(dataset, f, indent=2)

In [16]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
  Please solve the following task:

  {test_case["task"]}
"""
    
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [37]:
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [34]:
def run_test_case(test_case):
  """Calls run_prompt, then grades the result"""
  output = run_prompt(test_case)

  # TODO - Grading
  model_grade = grade_by_model(test_case, output)
  score = model_grade["score"]
  reasoning = model_grade["reasoning"]


  return {
    "output": output,
    "test_case": test_case,
    "score": score,
    "reasoning": reasoning
  }

In [40]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [41]:
with open("aws_prompt_evaluation_dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 7.333333333333333


In [39]:
print(json.dumps(results, indent=2))

[
  {
    "output": "# AWS S3 Bucket Name Validator\n\n```python\nimport re\n\ndef is_valid_s3_bucket_name(bucket_name: str) -> bool:\n    \"\"\"\n    Validates if a string is a valid AWS S3 bucket name according to AWS naming rules.\n    \n    Rules:\n    - Length: 3-63 characters\n    - Characters: lowercase letters (a-z), numbers (0-9), and hyphens (-)\n    - Must start with a lowercase letter or number\n    - Must end with a lowercase letter or number\n    - Cannot contain consecutive hyphens (optional but good practice)\n    \n    Args:\n        bucket_name (str): The bucket name to validate\n        \n    Returns:\n        bool: True if valid, False otherwise\n    \"\"\"\n    \n    # Check if bucket_name is a string\n    if not isinstance(bucket_name, str):\n        return False\n    \n    # Check length (3-63 characters)\n    if len(bucket_name) < 3 or len(bucket_name) > 63:\n        return False\n    \n    # Check if starts with lowercase letter or number\n    if not bucket_nam